 <div style="padding: 10px; margin-bottom: 10px; border: 1px solid #333333;">
    
# support mission [ラベル付け画像に基づく機械学習モデルの訓練]
    
- このsupport mission では、data_collectionで集めた画像をもとに入力されたデータが

## 完了の定義：

- このファイルと同じディレクトリ内にdataset、その下にfree、blockedディレクトリが作成されていること

</div>


## <span style="color: red; ">NEW</span> タグ：[ラベル付けに基づく機械学習モデルの訓練]

このノートブックでは、2つのクラスを検出するように画像分類器をトレーニングします

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

### データセットインスタンスの作成

ここで、集めた画像データを正規化します

In [2]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

### データの分割

次に、データセットをトレーニング用セットおよび、精度の確認用のテスト用セットに分割します。

In [3]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50])

### DataLoaderインスタンスの作成

2つのDataLoaderインスタンスを作成します。これらのインスタンスは、データのシャッフル、ミニバッチの生成などを行います

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### ニューラルネットワークを定義する

次に、トレーニングするニューラルネットワークを定義します。

torchvisionパッケージは、使用可能な事前トレーニング済みモデルを提供してくれます。

転送学習と呼ばれるプロセスでは、利用可能なデータがはるかに少ない可能性がある新しいタスクのために、事前にトレーニングされたモデル（数百万の画像でトレーニングされた）を再利用できます。

事前トレーニング済みモデルの元のトレーニングで学習した重要な機能は、新しいタスクに再利用できます。

今回はalexnetモデルを使用します。

In [5]:
model = models.alexnet(pretrained=True)

alexnetモデルは元々、クラスラベルが1000個あるデータセット用にトレーニングされていましたが、今回のデータセットにはクラスラベルが2つしかありません！ 最終レイヤーを出力が2つしかない新しいレイヤーに置き換えます。

In [6]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)


最後に、GPUで実行するためにモデルを転送します

In [7]:
device = torch.device('cuda')
model = model.to(device)

### トレーニング

以下のコードを使用して、30エポックのニューラルネットワークをトレーニングし、各エポックの後に最高のパフォーマンスモデルを保存します。

エポックとは、データを完全に把握することです。

In [8]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

RuntimeError: cublas runtime error : the GPU program failed to execute at /home/nvidia/Downloads/tmp/pytorch/aten/src/THC/THCBlas.cu:258

これが完了すると、Jupyter Labファイルブラウザーにファイルbest_model.pthが表示されます。